In [4]:
import glob
import Image
import numpy as np

import tensorflow as tf

In [5]:
def load_img(fpath):
    return np.array(Image.open(fpath))

jpg_files = glob.glob('*.jpg')

image_h = 300
image_w = 500
image_c = 3

In [3]:
images = []
for jpg_file in jpg_files:
    try:
        img = load_img(jpg_file)
    except :
        continue
    if img.ndim != 3 :
        continue
    if img.shape[0] < image_h or img.shape[1] < image_w or img.shape[2] < image_c :
        continue
    images.append(img[:image_h, :image_w, :image_c])

In [4]:
# This is how we can create an example protocol buffer from a numpy array

# Note that we can pass multiple arrays for an example - e.g. one could be features and one could be labels

examples = []
for img in images:
    examples.append(tf.train.Example(features=tf.train.Features(feature={
        'image': tf.train.Feature(
            float_list=tf.train.FloatList(value=img.flatten().astype(float)))
    })))

In [3]:
# This is how we can serialize protocol buffers and write them to disk

shard_size = 32

for i in range(0, len(examples), shard_size):
    fname = 'images-{}.tfrecords'.format(i)
    with tf.python_io.TFRecordWriter(fname) as writer:
        for example in examples[i:i+shard_size]:
            serialized = example.SerializeToString()
            writer.write(serialized)

NameError: name 'examples' is not defined

In [6]:
# create a string tensor with a list of our tfrecord files

fnames = tf.train.match_filenames_once('images-*.tfrecords')


# create a queue of filenames so we can iterate through them

fnames_queue = tf.train.string_input_producer(fnames, capacity=128)

num_readers = 4
enqueue_ops = []
features = []

image_shape = (image_h, image_w, image_c)

d_queue = tf.FIFOQueue(1000, dtypes=[tf.float32], shapes=[tf.TensorShape(image_shape)])

for _ in range(num_readers):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(fnames_queue)
    feature_defs = {'image': tf.FixedLenFeature(image_shape, tf.float32)}
    feature = tf.parse_single_example(serialized_example, features=feature_defs)['image']
    features.append(feature)
    enqueue_op = d_queue.enqueue(feature)
    enqueue_ops.append(enqueue_op)

q_runner = tf.train.QueueRunner(d_queue, enqueue_ops=enqueue_ops)

dequeued_examples = d_queue.dequeue_many(100)

In [7]:
sess = tf.InteractiveSession()

In [8]:
sess.run(tf.initialize_all_variables())

tf.train.start_queue_runners(sess)

[<Thread(Thread-4, started daemon 123145334603776)>]

In [ ]:
# Dequeue 100 examples into a batch:
dequeued_examples.eval()

1

In [ ]:
562500